<a href="https://colab.research.google.com/github/katyachernov/system-design-primer/blob/master/pandid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
hristohristov21_pid_symbols_path = kagglehub.dataset_download('hristohristov21/pid-symbols')
shrekty_pandid_trained_path = kagglehub.dataset_download('shrekty/pandid-trained')

print('Data source import complete.')


In [ ]:
import pandas as pd
import numpy as np
import torch
import torchvision
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch.optim as optim
import glob
import cv2

In [ ]:
df = pd.read_csv("/kaggle/input/pid-symbols/train (2).txt", names = ["path"]).reset_index(drop = True)
len(df)

In [ ]:
df["fnames"] = df["path"].str.split("/").str[2]
df["fnames"] = df["fnames"].str.split(".").str[0]

In [ ]:
df

In [ ]:
x = []
with open(f'/kaggle/input/pid-symbols/labels (2)/{df.fnames[4289]}.txt') as f:
    x.append([line[:-2] for line in f.readlines()])


In [ ]:
x

In [ ]:
[i.split(" ") for i in x[0]]

In [ ]:
x = [[int(float(val)) if float(val) == int(float(val)) else float(val) for val in i.split()]for i in x[0]]
x

In [ ]:
from PIL import Image
import numpy as np

img = Image.open(f"/kaggle/input/pid-symbols/images (3)/{df.fnames[4289]}.jpg")
img = img.resize((640,640))
print(img.size)
img = np.array(img).astype(np.float32)
img/=255

In [ ]:
img.shape[0]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.patches as patches
fig, ax = plt.subplots()
for box in x:
    s = img.shape
    print((box[1] - box[3]/2) * s[0], (box[2] - box[4]/2) * s[1])
    bb = patches.Rectangle(
        ((box[1] - box[3]/2) * s[0], (box[2] - box[4]/2) * s[1]),
        box[3] * s[0],box[4] * s[1],
        edgecolor="blue")
    ax.add_patch(bb)

ax.imshow(img)
plt.show()

In [ ]:
trans = T.Compose([T.ToTensor()])

class pidData(Dataset):
    def __init__(self, df, img_dir, ann_dir, img_size, trans = None, train = True):
        self.df = df
        self.transform = trans
        self.train = train
        self.ann_dir = ann_dir
        self.img_dir = img_dir
        self.img_size = img_size

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img = Image.open(f"{self.img_dir+self.df[idx]}.jpg").convert("RGB")
        img = img.resize((self.img_size, self.img_size))
        img = np.array(img).astype(np.float32)
        img/=255
        if self.transform is not None:
            img = self.transform(img)
        if (self.train == False):
            return img

        ann = []
        with open(f'{self.ann_dir+self.df[idx]}.txt') as f:
            ann.append([line[:-2] for line in f.readlines()])
        ann = [[int(float(val)) if float(val) == int(float(val)) else float(val) for val in i.split()]for i in ann[0]]
        boxes = []
        for i in ann:
            if i[3] <= 0 or i[4] <= 0:
                continue
            tmp = []
            tmp.append(float((i[1] - i[3]/2)*self.img_size))
            tmp.append(float((i[2] - i[4]/2)*self.img_size))
            tmp.append(float((i[1] + i[3]/2)*self.img_size))
            tmp.append(float((i[2] + i[4]/2)*self.img_size))
            boxes.append(tmp)
            tmp = []
        target = {}
        target["boxes"] = torch.as_tensor(boxes, dtype = torch.float32)
        target["labels"] = torch.as_tensor([j[0] for j in ann], dtype=torch.int64)


        return img, target

In [ ]:
x = []
empty_files = []
for i in range(len(df)):
    with open(f'/kaggle/input/pid-symbols/labels (2)/{df.fnames[i]}.txt') as f:
        x.append([line[:-2] for line in f.readlines()])
    if len(x[0]) == 0:
        empty_files.append(df.fnames[i])
    x = []

In [ ]:
df = df[~df['fnames'].isin(empty_files)]

In [ ]:
ex = df.sample(5).fnames.values
ex

In [ ]:
samp = pidData(list(ex), "/kaggle/input/pid-symbols/images (3)/", "/kaggle/input/pid-symbols/labels (2)/", 320, trans = trans)

In [ ]:
def custom_collate(batch):
    return tuple(zip(*batch))

In [ ]:
dl = DataLoader(samp, batch_size=5, collate_fn=custom_collate)

In [ ]:
# batch = next(iter(dl))
X, y = next(iter(dl))

In [ ]:
X[0].shape, y[0]["boxes"].shape, y[0]["labels"].shape

In [ ]:
y[0]

In [ ]:
fig, ax = plt.subplots()
image = X[0].permute(1, 2, 0).detach().numpy()
image_with_rect = image.copy()

for x in y[0]['boxes']:
    cv2.rectangle(image_with_rect, (int(x[0].numpy()), int(x[1].numpy())), (int(x[2].numpy()), int(x[3].numpy())), (222, 0, 0), 1)

ax.imshow(image_with_rect)
plt.show()

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True, pretrained_backbone=True)

In [ ]:
train = df.head(500).fnames.values

In [ ]:
len(train)

In [ ]:
train_ds = pidData(list(train), "/kaggle/input/pid-symbols/images (3)/", "/kaggle/input/pid-symbols/labels (2)/", 640, trans = trans)
train_dl = DataLoader(train_ds, batch_size=5, collate_fn=custom_collate, shuffle=False)

In [ ]:
X, y = next(iter(train_dl))

In [ ]:
X[0].shape, y[0]["boxes"].shape, y[0]["labels"].shape

In [ ]:
num_classes = 33
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
optimizer = optim.SGD(model.parameters(), lr = 2e-4, momentum=0.9, weight_decay=1e-4)
num_epochs = 30

In [ ]:
checkpoint = torch.load('/kaggle/input/pandid-trained/pandid_rcnn.t7')
model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])
epochs = checkpoint['epoch']

In [ ]:
# for images, targets in train_dl:
#     if str(targets[0]["boxes"].shape) == "torch.Size([0])":
#         print(targets)
#         break

In [ ]:
itr = 1
model.to(device)
model.train()
for epoch in range(num_epochs):
    epoch_loss = 0
    for images, targets in train_dl:

        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)   ##Return the loss

        losses = sum(loss for loss in loss_dict.values())
        epoch_loss += losses.cpu().detach().numpy()
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if itr % 50 == 0:
            print(f"Iteration #{itr} loss: {loss_value}")

        itr += 1


    print(f"Epoch #{epoch} loss: {epoch_loss}")

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
val = [df.iloc[1001].fnames]
val_ds = pidData(val, "/kaggle/input/pid-symbols/images (3)/", "/kaggle/input/pid-symbols/labels (2)/", 320, trans = trans)
val_dl = DataLoader(val_ds, batch_size=1, shuffle=False)

In [ ]:
X, y= next(iter(val_dl))

In [ ]:
X.shape

In [ ]:
y

In [ ]:
model.to(device)
model.eval()
out = model(list(X.to(device)))

In [ ]:
out_boxes = out[0]["boxes"]
out_labels = out[0]["labels"]
out_scores = out[0]["scores"]

In [ ]:
out_scores

In [ ]:
out_boxes = out_boxes[out_scores >= 0.45].data.cpu().numpy().astype(np.int32)
out_labels = out_labels[out_scores >= 0.45].data.cpu().numpy().astype(np.int32)

In [ ]:
out_boxes

In [ ]:
out_labels

In [ ]:
y

In [ ]:
og_boxes  = y["boxes"].squeeze().data.cpu().numpy().astype(np.int32)
og_labels  = y["labels"].squeeze().data.cpu().numpy().astype(np.int32)

In [ ]:
og_boxes

In [ ]:
def generate_color_map(num_classes):
    colors = []
    for i in range(num_classes):
        hue = i / num_classes
        rgb = plt.cm.hsv(hue)[:3]
        colors.append(tuple(np.array(rgb) * 255))
    return colors

num_classes = 33
class_colors = generate_color_map(num_classes)


fig, ax = plt.subplots()

image = X[0].permute(1, 2, 0).detach().numpy()
image_with_rect = image.copy()
for x, label in zip(out_boxes, out_labels):
    x_min, y_min, x_max, y_max = x
    color = class_colors[label.item()]
    color = tuple(c / 255.0 for c in color)
    cv2.rectangle(image_with_rect, (x_min, y_min), (x_max, y_max), color, 1)

    text = f"Class: {label.item()}"
    ax.text(x_min, y_min - 5, text, color='white', fontsize=8, weight='bold',
            bbox=dict(facecolor=color, edgecolor=color, pad=0, alpha=0.7))

ax.imshow(image_with_rect)
plt.axis('off')
plt.show()

In [ ]:
fig, ax = plt.subplots()

image = X[0].permute(1, 2, 0).detach().numpy()
image_with_rect = image.copy()
for x, label in zip(og_boxes, og_labels):
    x_min, y_min, x_max, y_max = x
    color = class_colors[label.item()]
    color = tuple(c / 255.0 for c in color)
    cv2.rectangle(image_with_rect, (x_min, y_min), (x_max, y_max), color, 1)

    text = f"Class: {label.item()}"
    ax.text(x_min, y_min - 5, text, color='white', fontsize=8, weight='bold',
            bbox=dict(facecolor=color, edgecolor=color, pad=0, alpha=0.7))

ax.imshow(image_with_rect)
plt.axis('off')
plt.show()

In [ ]:
state = {
        'epoch': num_epochs,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
        }
savepath = './pandid_rcnn.t7'
torch.save(state,savepath)

In [ ]:
# fig, ax = plt.subplots()
# image = X[0].permute(1, 2, 0).detach().numpy()
# image_with_rect = image.copy()

# for x in out_boxes:
#     cv2.rectangle(image_with_rect, (x[0], x[1]), (x[2], x[3]), (222, 0, 0), 1)

# ax.imshow(image_with_rect)
# plt.show()